In [1]:
CNY_USD = 0.1389
RUB_USD = 0.010309087

Today = '31.12.2023'

Group = 'SUEK'


Data_Forwards = 'WHWEEK_quantumForwards_2023_12' + '.xlsx'
Data_Swaps = 'WHWEEK_quantumSwaps_2023_12' + '.xlsx'
Sheet_in_data_file = 'Export Worksheet'

Mapping_file = 'C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP\\Методология\\Mapping.xlsx'
Sheet_in_mapping_file = 'mapping'

from datetime import date

Output_Forwards = "_".join([str(date.today()), Group, 'quantum', 'Forwards.xlsx'])
Output_Swaps = "_".join([str(date.today()), Group, 'quantum', 'Swaps.xlsx'])
Sheet_in_output_Deriv = 'Deriv'

In [3]:
import pandas as pd
import numpy as np

data_Forwards_export = pd.read_excel(Data_Forwards, sheet_name = Sheet_in_data_file)
data_Swaps_export = pd.read_excel(Data_Swaps, sheet_name = Sheet_in_data_file)

data_mapping = pd.read_excel(Mapping_file, sheet_name = Sheet_in_mapping_file)

c:\Users\KlimovaAnnaA\AppData\Local\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\KlimovaAnnaA\AppData\Local\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


ModuleNotFoundError: No module named 'Quantum_Debt'

In [40]:
data_Forwards_export['holdingEntity'] = data_Forwards_export['holdingEntity'].replace({'XXTSU': 'SUEK'})
data_Forwards_export = data_Forwards_export[data_Forwards_export['holdingEntity'] == Group].reset_index(drop=True)
data_Swaps_export['holdingEntity'] = data_Swaps_export['holdingEntity'].replace({'XXTSU': 'SUEK'})
data_Swaps_export = data_Swaps_export[data_Swaps_export['holdingEntity'] == Group].reset_index(drop=True)
# Знамена значений в стодбце holdingEntity и фильтр по Group

In [41]:
assert set(data_Swaps_export['dealNo']).intersection(set(data_Forwards_export['dealNo'])) == set()

In [42]:
Data_for_loop = data_Forwards_export[['entity','holdingEntity','counterparty','dealNo', 'payFaceValue', 'payFXCurrency', 'recFaceValue', 'recFXCurrency', 'maturityDate']]

for i in range(0, len(Data_for_loop)):
    row_list = list(Data_for_loop.loc[i])
    Data_for_loop.loc[len(Data_for_loop)] = row_list[:4] + row_list[6:8] + row_list[6:]
    Data_for_loop.loc[i,'payFaceValue'] = Data_for_loop.loc[i,'payFaceValue'] * -1
Data_for_loop = Data_for_loop.sort_values('dealNo')
# цикл создания новых строк

merge_df = Data_for_loop.merge(data_mapping, how = 'left', left_on = 'entity', right_on = 'Unnamed: 23')[['Unnamed: 24', 'Unnamed: 25']]
Data_for_loop = pd.concat([Data_for_loop, merge_df], axis=1).rename(columns={'Unnamed: 24': 'Entity', 'Unnamed: 25': 'Entity_group'})
# Merge столбца entity по XYZ в mapping

day = pd.to_datetime(Today, dayfirst = True)
Data_for_loop['Days'] = 0

Data_for_loop['Days'] = Data_for_loop.loc[Data_for_loop['maturityDate'] != np.NaN, 'maturityDate'] - day
Data_for_loop['Days'] = Data_for_loop['Days'].dt.days
Data_for_loop['Period'] = '2Y+'

Data_for_loop.loc[pd.to_numeric(Data_for_loop['Days']) < 730, 'Period'] = '1Y-2Y'
Data_for_loop.loc[pd.to_numeric(Data_for_loop['Days']) < 365, 'Period'] = '6M-1Y'
Data_for_loop.loc[pd.to_numeric(Data_for_loop['Days']) < 182, 'Period'] = '3M-6M'
Data_for_loop.loc[pd.to_numeric(Data_for_loop['Days']) < 91, 'Period'] = '1M-3M'
Data_for_loop.loc[pd.to_numeric(Data_for_loop['Days']) < 30, 'Period'] = '<1M'
# Создание Days и Period

Data_for_loop['USD_equiv'] = 0
for i in range (0,len(Data_for_loop)):
    if Data_for_loop.loc[i, 'payFXCurrency'] == 'CNY':
        Data_for_loop.loc[i, 'USD_equiv'] = Data_for_loop.loc[i, 'payFaceValue'] * CNY_USD
    elif Data_for_loop.loc[i, 'payFXCurrency'] == 'RUB':
        Data_for_loop.loc[i, 'USD_equiv'] = Data_for_loop.loc[i, 'payFaceValue'] * RUB_USD
    elif Data_for_loop.loc[i, 'payFXCurrency'] == 'USD':
        Data_for_loop.loc[i, 'USD_equiv'] = Data_for_loop.loc[i, 'payFaceValue']
# создание USD_equiv

new_columns = ['Company', 'Settlement', 'Notional Amount (USD)', 'Unnamed', 'source']
Data_for_loop = Data_for_loop.reindex(columns=(Data_for_loop.columns.tolist() + new_columns))
Data_for_loop['source'] = 'Quantum'
# Создание пустых столбцов и столбца ресурса

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9256\474526847.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data_for_loop.loc[len(Data_for_loop)] = row_list[:4] + row_list[6:8] + row_list[6:]
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9256\474526847.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data_for_loop.loc[len(Data_for_loop)] = row_list[:4] + row_list[6:8] + row_list[6:]
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9256\474526847.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '21676144.053375' has

In [43]:
Data_for_loop_Swap = data_Swaps_export[['entity','holdingEntity','counterparty','dealNo', 'payFaceValue', 'payFXCurrency', 'recFaceValue', 'recFXCurrency', 'maturityDate']]

for i in range(0, len(Data_for_loop_Swap)):
    row_list = list(Data_for_loop_Swap.loc[i])
    Data_for_loop_Swap.loc[len(Data_for_loop_Swap)] = row_list[:4] + row_list[6:8] + row_list[6:]
    Data_for_loop_Swap.loc[i,'payFaceValue'] = Data_for_loop_Swap.loc[i,'payFaceValue'] * -1
Data_for_loop_Swap = Data_for_loop_Swap.sort_values('dealNo')
# цикл создания новых строк

merge_df = Data_for_loop_Swap.merge(data_mapping, how = 'left', left_on = 'entity', right_on = 'Unnamed: 23')[['Unnamed: 24', 'Unnamed: 25']]
Data_for_loop_Swap = pd.concat([Data_for_loop_Swap, merge_df], axis=1).rename(columns={'Unnamed: 24': 'Entity', 'Unnamed: 25': 'Entity_group'})
# Merge столбца entity по XYZ в mapping

day = pd.to_datetime(Today, dayfirst = True)
Data_for_loop_Swap['Days'] = 0

Data_for_loop_Swap['Days'] = Data_for_loop_Swap.loc[Data_for_loop_Swap['maturityDate'] != np.NaN, 'maturityDate'] - day
Data_for_loop_Swap['Days'] = Data_for_loop_Swap['Days'].dt.days
Data_for_loop_Swap['Period'] = '2Y+'

Data_for_loop_Swap.loc[pd.to_numeric(Data_for_loop_Swap['Days']) < 730, 'Period'] = '1Y-2Y'
Data_for_loop_Swap.loc[pd.to_numeric(Data_for_loop_Swap['Days']) < 365, 'Period'] = '6M-1Y'
Data_for_loop_Swap.loc[pd.to_numeric(Data_for_loop_Swap['Days']) < 182, 'Period'] = '3M-6M'
Data_for_loop_Swap.loc[pd.to_numeric(Data_for_loop_Swap['Days']) < 91, 'Period'] = '1M-3M'
Data_for_loop_Swap.loc[pd.to_numeric(Data_for_loop_Swap['Days']) < 30, 'Period'] = '<1M'
# Создание Days и Period

Data_for_loop_Swap['USD_equiv'] = 0
for i in range (0,len(Data_for_loop_Swap)):
    if Data_for_loop_Swap.loc[i, 'payFXCurrency'] == 'CNY':
        Data_for_loop_Swap.loc[i, 'USD_equiv'] = Data_for_loop_Swap.loc[i, 'payFaceValue'] * CNY_USD
    elif Data_for_loop_Swap.loc[i, 'payFXCurrency'] == 'RUB':
        Data_for_loop_Swap.loc[i, 'USD_equiv'] = Data_for_loop_Swap.loc[i, 'payFaceValue'] * RUB_USD
    elif Data_for_loop_Swap.loc[i, 'payFXCurrency'] == 'USD':
        Data_for_loop_Swap.loc[i, 'USD_equiv'] = Data_for_loop_Swap.loc[i, 'payFaceValue']
# создание USD_equiv

new_columns = ['Company', 'Settlement', 'Notional Amount (USD)', 'Unnamed', 'source']
Data_for_loop_Swap = Data_for_loop_Swap.reindex(columns=(Data_for_loop_Swap.columns.tolist() + new_columns))
Data_for_loop_Swap['source'] = 'Quantum'
# Создание пустых столбцов и столбца ресурса

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9256\1935957059.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data_for_loop_Swap.loc[len(Data_for_loop_Swap)] = row_list[:4] + row_list[6:8] + row_list[6:]
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9256\1935957059.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data_for_loop_Swap.loc[len(Data_for_loop_Swap)] = row_list[:4] + row_list[6:8] + row_list[6:]
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9256\1935957059.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentatio

In [44]:
data = Data_for_loop[['Entity','Entity_group','holdingEntity','Company',\
                    'counterparty','dealNo','payFXCurrency','payFaceValue',\
                    'USD_equiv','Settlement','Notional Amount (USD)',\
                    'Unnamed','maturityDate','Days','Period','source']]
data_Swap = Data_for_loop_Swap[['Entity','Entity_group','holdingEntity','Company',\
                    'counterparty','dealNo','payFXCurrency','payFaceValue',\
                    'USD_equiv','Settlement','Notional Amount (USD)',\
                    'Unnamed','maturityDate','Days','Period','source']]
# Выбор столбцов

In [56]:
data_to_excel = pd.concat([data, data_Swap], axis=0)

In [57]:
data_to_excel.to_excel(Output_Forwards, sheet_name = Sheet_in_output_Deriv, index = False)
# Запись в файл 